Automatic classification of stigmatizing mental illness articles in online news journals - April 2022
Author: Alina Yanchuk - alinayanchuk@ua.pt

### Table of contents:

* [3. Classification](#chapter3)
    * [3.1 Requirements](#section_3_1)
    * [3.2 Imports](#section_3_2)
    * [3.3 Get data](#section_3_3)
    * [3.4 Train and Test dataset](#section_3_4)
    * [3.5 Generate feature vectors](#section_3_5)
    * [3.6 Models training and optimization (Machine Learning)](#section_3_6)
        * [3.6.1 Logistics Regression](#section_3_6_1)
            * [3.6.1.1 With bag-of-words](#section_3_6_1_1)
            * [3.6.1.2 With TF-IDF](#section_3_6_1_2)
            * [3.6.1.3 With LIWC](#section_3_6_1_3)
        * [3.6.2 Linear Support Vector Classifier (SVC)](#section_3_6_2)
            * [3.6.2.1 With bag-of-words](#section_3_6_2_1)
            * [3.6.2.2 With TF-IDF](#section_3_6_2_2)
            * [3.6.2.3 With LIWC](#section_3_6_2_3)
        * [3.6.3 Multinomial Naive Bayes](#section_3_6_3)
            * [3.6.3.1 With bag-of-words](#section_3_6_3_1)
            * [3.6.3.2 With TF-IDF](#section_3_6_3_2)
            * [3.6.3.3 With LIWC](#section_3_6_3_3)
        * [3.6.4 K-Nearest Neighbors](#section_3_6_4)
            * [3.6.4.1 With bag-of-words](#section_3_6_4_1)
            * [3.6.4.2 With TF-IDF](#section_3_6_4_2)
            * [3.6.4.3 With LIWC](#section_3_6_4_3)
        * [3.6.5 Random Forest](#section_3_6_5)
            * [3.6.5.1 With bag-of-words](#section_3_6_5_1)
            * [3.6.5.2 With TF-IDF](#section_3_6_5_2)
            * [3.6.5.3 With LIWC](#section_3_6_5_3)
    * [3.7 Models training and optimization (Deep learning)](#section_3_7)
        * [3.7.1 Convolutional Neural Network](#section_3_7_1)
        * [3.7.2 Long Short-Term Memory (LSTM)](#section_3_7_2)
        * [3.7.3 Bidirectional Long Short-Term Memory (Bi-LSTM)](#section_3_7_3)
    * [3.8 Evaluation analysis](#section_3_8)

# 3. Classification <a class="anchor" id="chapter3"></a>

Text classification is a problem that belongs to the category of algorithms that
use supervised learning and consists of an automatic process of associating data
texts to a given class. 

In this step, a classification task is performed using some of the most popular Machine Learning and Deep Learning algorithms for binary text classification. 

## 3.1 Requirements <a class="anchor" id="section_3_1"></a>

In [ ]:
#pip install scikit-optimize

In [ ]:
#pip install tensorflow 

In [ ]:
#pip install tensorflow-gpu 

In [40]:
#pip install keras-tuner

In [1]:
#pip install pydot
#pip install graphviz

In [2]:
#pip install tensorflow_hub

## 3.2 Imports <a class="anchor" id="section_3_2"></a>

In [2]:
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split

from math import floor, log

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from keras.preprocessing import text, sequence
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as scores

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Conv1D, Flatten, MaxPooling1D, LSTM, Bidirectional

from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.utils import use_named_args

import keras_tuner as kt

2022-04-22 22:01:48.871525: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-22 22:01:48.871600: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/alina/anaconda3/envs/stigma/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## 3.3 Get data <a class="anchor" id="section_3_3"></a>

In [3]:
data = pd.read_pickle('data_preprocessed.pkl')
data.head()

,label,content
0,0,prisão perpétua homem tentou assassinar senado...
1,0,john nash matemático mente brilhante morre aci...
2,1,mito reeleição mínima garantida cavaco sairá d...
3,0,morreu rita levintalcini grande dama ciência i...
4,0,trás porta amarela homem problemas psicológico...


## 3.4 Train and Test dataset <a class="anchor" id="section_3_4"></a>

In [4]:
# Divide the data into a 80% train dataset and 20% test dataset

X = data.loc[:,'content']
y = data.loc[:,'label']

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X, y, data.index, test_size=0.2, random_state=55)

print("Number of news in train dataset: " + str(len(X_train)))
print("Number of news in test dataset: " + str(len(X_test)))

Number of news in train dataset: 782
Number of news in test dataset: 196


## 3.5 Generate feature vectors <a class="anchor" id="section_3_5"></a>

In [5]:
# Generate bag-of-words feature vectors

bow_vectorizer = CountVectorizer(lowercase=False)
bow_train = bow_vectorizer.fit_transform(X_train)
bow_test = bow_vectorizer.transform(X_test)

bow_train = (bow_train - np.min(bow_train)) / (np.max(bow_train) - np.min(bow_train)) # Normalize to range [0, 1]
bow_test = (bow_test - np.min(bow_test)) / (np.max(bow_test) - np.min(bow_test))

# (Number of news, Number of features/unique words in training dataset)
bow_train.shape

# IF SAVED - Load saved vectorizer
#bow_vectorizer = pickle.load(open("vectors/BOW.pickle", 'rb'))

(782, 44522)

In [6]:
# Generate TF-IDF feature vectors
# "words that are unique to particular document would have higher weights compared to words that are used commonly across documents"

tfidf_vectorizer = TfidfVectorizer(lowercase=False,)
tfidf_train = tfidf_vectorizer.fit_transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)

# (Number of news, Number of features/unique words in training dataset)
tfidf_train.shape

# IF SAVED - Load saved vectorizer
#tfidf_vectorizer = pickle.load(open("vectors/TFIDF.pickle", 'rb'))

(782, 44522)

In [7]:
# Generate LIWC-PT feature vectors
# "lexicon that assigns words to categories"
# http://143.107.183.175:21380/portlex/index.php/pt/projetos/liwc
# https://lit.eecs.umich.edu/geoliwc/liwc_dictionary.html

liwc_dictionary = {}
# IF SAVED - Load saved mapping
#liwc_dictionary = pickle.load(open("vectors/LIWC.pickle", 'rb'))


# Read the file and create the LIWC dictionary mapping (word-categories) (If not in repo, need to download from official website above)
with open("dictionary/liwc.txt", "r", encoding = "ISO-8859-1") as liwc_file:
    start = False
    for line in liwc_file:
        try:
            line = line.split()
            if start==False:
                if line[0]=="a":
                    start=True
                    word = line[0]
                    vector = np.asarray(line[1:], dtype='float32')
                    liwc_dictionary[word] = vector
            else:
                word = line[0]
                vector = np.asarray(line[1:], dtype='float32')
                liwc_dictionary[word] = vector
        except:
            pass

liwc_file.close()

# Create the feature vectors: each vector will have all the LIWC categories and the corresponding values will be incremented everytime a word in the news article (document) fits in that category

documents_train = list(X_train.values.flatten()) # List of all news
documents_train = [string.split() for string in documents_train] # Split each news article in tokens/words

liwc_train = np.zeros((len(X_train), 464))
for document in documents_train:
    i = documents_train.index(document)
    for word in document:
        if word in liwc_dictionary:
            vector = liwc_dictionary[word]
            for index in vector:
                liwc_train[i][int(index)-1] = liwc_train[i][int(index)-1] + 1

documents_test = list(X_test.values.flatten())
documents_test = [string.split() for string in documents_test]

liwc_test = np.zeros((len(X_test), 464))
for document in documents_test:
    i = documents_test.index(document)
    for word in document:
        if word in liwc_dictionary:
            vector = liwc_dictionary[word]
            for index in vector:
                liwc_test[i][int(index)-1] = liwc_test[i][int(index)-1] + 1

liwc_train = (liwc_train - np.min(liwc_train)) / (np.max(liwc_train) - np.min(liwc_train)) # Normalize to range [0, 1]
liwc_test = (liwc_test - np.min(liwc_test)) / (np.max(liwc_test) - np.min(liwc_test)) 

# (Number of news, Number of features/categories in training dataset)
liwc_train.shape

(782, 464)

In [8]:
# Generate Word Embeddings vectors and matrix
# Glove PT 300D from NILC-Embeddings 
# http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

words_per_article = list(map(lambda x : len(x.split()),data.content))
avg = sum(words_per_article)/len(words_per_article) # Average number of words per news article
avg = pow(2, floor(log(avg)/log(2))) # Convert to nearest power of 2

results = Counter()
data['content'].str.lower().str.split().apply(results.update) # Number of unique words in all dataset

max_length_content = int(avg) # Max number of content that each object/news article will have after padding
vocabulary_length = len(results) # Max number of unique words

# Create word index
token = text.Tokenizer(lower=False, num_words=vocabulary_length) 
token.fit_on_texts(data['content']) # Tokenize all corpus
word_index = token.word_index # Index of unique words (dictionary)

# Convert text to sequence of tokens and pad them to ensure equal length vectors 
we_train = sequence.pad_sequences(token.texts_to_sequences(X_train), maxlen=max_length_content)
we_test = sequence.pad_sequences(token.texts_to_sequences(X_test), maxlen=max_length_content)

# Load the pre-trained word-embedding vectors (Glove 300D)
embedding_index = {}
# IF SAVED - Load saved mapping
#embedding_index = pickle.load(open("vectors/WE_glove.pickle", 'rb'))

# If not in repo, need to download from official website above
with open("pre-trained/glove_s300.txt", "r") as we_file:
    first_line = True
    for line in we_file:
        try:
            if first_line: # Ignore header
                first_line = False
            else:
                line = line.split()
                word = line[0]
                vector = np.asarray(line[1:], dtype='float32')
                embedding_index[word] = vector
        except:
            pass


# Create token-embedding mapping (with words from our dataset)
embedding_matrix = np.zeros((vocabulary_length, 300))
for word, i in word_index.items():
    if i > vocabulary_length - 1:
        break
    else:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

#(Number (vocabulary size) of features/unique words in all dataset, Number of dimensions/features)
embedding_matrix.shape

(51285, 300)

In [14]:
# Save feature vectorizers and mappings


with open('vectors/TFIDF.pickle', 'wb') as f1:
    pickle.dump(tfidf_vectorizer, f1)
f1.close()

with open('vectors/BOW.pickle', 'wb') as f2:
    pickle.dump(bow_vectorizer, f2)
f2.close()

with open('vectors/WE.pickle', 'wb') as f3:
    pickle.dump(embedding_index, f3)
f3.close()

with open('vectors/LIWC.pickle', 'wb') as f4:
    pickle.dump(liwc_dictionary, f4)
f4.close()


"\nwith open('vectors/TFIDF.pickle', 'wb') as f1:\n    pickle.dump(tfidf_vectorizer, f1)\nf1.close()\n\nwith open('vectors/BOW.pickle', 'wb') as f2:\n    pickle.dump(bow_vectorizer, f2)\nf2.close()\n\nwith open('vectors/WE.pickle', 'wb') as f3:\n    pickle.dump(embedding_index, f3)\nf3.close()\n\nwith open('vectors/LIWC.pickle', 'wb') as f4:\n    pickle.dump(liwc_dictionary, f4)\nf4.close()\n"

## 3.6 Models training and optimization (Machine Learning) <a class="anchor" id="section_3_6"></a>

#### Training

Attempt to get the best combination of weights and bias to the model to minimize the loss function over the prediction range and incrementally improve model's ability to predict the classes.

#### Optimization

Hyper-parameters are parameters that are not directly learnt within estimators. In scikit-learn they are passed as arguments to the constructor of the estimator classes. They define how our model is actually structured. Hyper-parameter tuning is just an optimization loop on top of ML model learning to find the set of hyper-parameters leading to the lowest error on the validation set.

-  The optimization for the classic Machine Learning algorithms is performed with the K-fold cross (K=5) validation startegy, in order to essentially combine training and validation data for both learning the model parameters and evaluating the model without introducing data leakage.

    Library used -> Scikit-optimize: uses a Sequential model-based optimization algorithm to find optimal solutions for hyperparameter search problems in less time. 
    
    Reference: https://scikit-optimize.github.io/stable/auto_examples/hyperparameter-optimization.html

- Optimization for the Deep Learning algorithms:

    Library used -> Keras Tuner: The Hyperband algorithm is a variation of random search, but with some explore-exploit theory to find the best time allocation for each of the configurations.
    
    Reference: https://www.tensorflow.org/tutorials/keras/keras_tuner


In [9]:
# Create list to store evaluation metrics for all models

evaluation_metrics = []

# Function to store evaluation metrics for a model

def evaluation(evaluation_metrics_list, model, model_name, test_features, test_labels, feature_vectorizer_name, deep_learning=False):

    y_pred = model.predict(test_features)

    if deep_learning:
        y_pred = [int(round(p[0])) for p in y_pred]

    # Performance metrics
    accuracy = accuracy_score(test_labels, y_pred)*100

    # Precision, recall, f1 scores
    precision, recall, f1score, support = scores(y_test, y_pred, average='micro')

    # Add metrics to evaluation list
    evaluation_metrics_list.append(dict([
        ('Model', model_name),
        ('Feature Vectorizer', feature_vectorizer_name),
        ('Accuracy (%)', round(accuracy, 2)),
        ('Precision', round(precision, 2)),
        ('Recall', round(recall, 2)),
        ('F1', round(f1score, 2))
    ]))

    return evaluation_metrics_list

In [10]:
# Function to predict labels (using the trained model) to a dataset

def prediction(trained_model, features):

    y_pred = trained_model.predict(features)

    return y_pred

In [11]:
# Function to insert predicted labels to original data's csv file

def insert(pred_labels, file_name):

    original_data = pd.read_pickle(file_name)

    original_data.insert('Predicted label', pred_labels)

    original_data.to_csv("result_" + file_name, index=False)

### 3.6.1 Logistic Regression <a class="anchor" id="section_2_6_1"></a>

- Is a classical linear method for binary classification (fits a line to best separate the two classes);
- It can handle both dense and sparse input;


#### 3.6.1.1 With bag-of-words <a class="anchor" id="section_3_6_1_1"></a>

In [19]:
# Model 

model_lr_bow = LogisticRegression(random_state=0) # solver: algorithm to use in the optimization problem (liblinear for small datasets)
#IF SAVED - Load saved model
#model_lr_bow = pickle.load(open("models/LogisticRegressionBOW.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Real(0, 10, name='C')] # Inverse of regularization strength

@use_named_args(space)
def objective(**params):
    model_lr_bow.set_params(**params)

    return -np.mean(cross_val_score(model_lr_bow, bow_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_C_lr_bow = res_gp.x[0]

print("""Best parameters: - C=%f""" % (best_C_lr_bow))

Best parameters: - C=9.367025


In [12]:
# Model training with BEST hyper-parameteres

model_lr_bow = LogisticRegression(random_state=0, C=best_C_lr_bow, solver="liblinear").fit(bow_train, y_train)

# Model saving
with open('models/LogisticRegressionBOW.pickle', 'wb') as f:
    pickle.dump(model_lr_bow, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_lr_bow, model_name="Logistic Regression", test_features=bow_test, test_labels=y_test, feature_vectorizer_name="Bag-of-words")

print("Logistic Regression (Bag-of-words) successfully trained and stored.")

Logistic Regression (Bag-of-words) successfully trained and stored.


#### 3.6.1.2 With TF-IDF <a class="anchor" id="section_3_6_1_2"></a>

In [21]:
# Model 

model_lr_tfidf = LogisticRegression(random_state=0)
# IF SAVED - Load saved model
#model_lr_tfidf = pickle.load(open("models/LogisticRegressionTFIDF.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Real(0, 10, name='C')] # Inverse of regularization strength

@use_named_args(space)
def objective(**params):
    model_lr_tfidf.set_params(**params)

    return -np.mean(cross_val_score(model_lr_tfidf, tfidf_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_C_lr_tfidf = res_gp.x[0]

print("""Best parameters: - C=%f""" % (best_C_lr_tfidf))

Best parameters: - C=3.843817


In [13]:
# Model training with BEST hyper-parameteres

model_lr_tfidf = LogisticRegression(random_state=0, C=best_C_lr_tfidf, solver="liblinear").fit(tfidf_train, y_train)

# Model saving
with open('models/LogisticRegressionTFIDF.pickle', 'wb') as f:
    pickle.dump(model_lr_tfidf, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_lr_tfidf, model_name="Logistic Regression", test_features=tfidf_test, test_labels=y_test, feature_vectorizer_name="TF-IDF")

print("Logistic Regression (TF-IDF) sucessfully trained and stored.")

Logistic Regression (TF-IDF) sucessfully trained and stored.


#### 3.6.1.3 With LIWC <a class="anchor" id="section_3_6_1_3"></a>

In [23]:
# Model 

model_lr_liwc = LogisticRegression(random_state=0)
# IF SAVED - Load saved model
#model_lr_LIWC = pickle.load(open("models/LogisticRegressionLIWC.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Real(0, 10, name='C')] # Inverse of regularization strength

@use_named_args(space)
def objective(**params):
    model_lr_liwc.set_params(**params)

    return -np.mean(cross_val_score(model_lr_liwc, liwc_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_C_lr_liwc = res_gp.x[0]

print("""Best parameters: - C=%f""" % (best_C_lr_liwc))

/home/alina/anaconda3/envs/stigma/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/alina/anaconda3/envs/stigma/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/alina/anaconda3/envs/stigma/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/alina/anaconda3/envs/stigma/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/alina/anaconda3/envs/stigma/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been eval

Best parameters: - C=9.694427


In [15]:
# Model training with BEST hyper-parameteres

model_lr_liwc = LogisticRegression(random_state=0, C=best_C_lr_liwc, solver="liblinear").fit(liwc_train, y_train)

# Model saving
with open('models/LogisticRegressionLIWC.pickle', 'wb') as f:
    pickle.dump(model_lr_liwc, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_lr_liwc, model_name="Logistic Regression", test_features=liwc_test, test_labels=y_test, feature_vectorizer_name="LIWC")

print("Logistic Regression (LIWC) sucessfully trained and stored.")

Logistic Regression (LIWC) sucessfully trained and stored.


### 3.6.2 Linear Support Vector Classifier (SVC) <a class="anchor" id="section_3_6_2"></a>

 - Supports both dense and sparse input 

#### 3.6.2.1 With bag-of-words <a class="anchor" id="section_3_6_2_1"></a>

In [25]:
# Model

model_svc_bow = LinearSVC(random_state=0)
# IF SAVED - Load saved model
#model_svc_bow = pickle.load(open("models/LinearSVCBOW.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Real(1, 15, name='C')] # Inverse of regularization strength

@use_named_args(space)
def objective(**params):
    model_svc_bow.set_params(**params)

    return -np.mean(cross_val_score(model_svc_bow, bow_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_C_svc_bow = res_gp.x[0]

print("""Best parameters: - C=%f""" % (best_C_svc_bow))

Best parameters: - C=12.819720


In [17]:
# Model training with BEST hyper-parameteres

model_svc_bow = LinearSVC(random_state=0, C=best_C_svc_bow).fit(bow_train, y_train)

# Model saving
with open('models/LinearSVCBOW.pickle', 'wb') as f:
    pickle.dump(model_svc_bow, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_svc_bow, model_name="Support Vector Classifier (Linear)", test_features=bow_test, test_labels=y_test, feature_vectorizer_name="Bag-of-words")

print("Linear Suppport Vector Classifier (Bag-of-words) successfully trained and stored.")

Linear Suppport Vector Classifier (Bag-of-words) successfully trained and stored.


#### 3.6.2.2 With TF-IDF <a class="anchor" id="section_3_6_2_2"></a>

In [27]:
# Model

model_svc_tfidf = LinearSVC(random_state=0)
# IF SAVED - Load saved model
#model_svc_tfidf = pickle.load(open("models/LinearSVCTFIDF.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Real(1, 15, name='C')] # Inverse of regularization strength

@use_named_args(space)
def objective(**params):
    model_svc_tfidf.set_params(**params)

    return -np.mean(cross_val_score(model_svc_tfidf, tfidf_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_C_svc_tfidf = res_gp.x[0]

print("""Best parameters: - C=%f""" % (best_C_svc_tfidf))

/home/alina/anaconda3/envs/stigma/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/alina/anaconda3/envs/stigma/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/alina/anaconda3/envs/stigma/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Best parameters: - C=9.299825


In [18]:
# Model training with BEST hyper-parameteres

model_svc_tfidf = LinearSVC(random_state=0, C=best_C_svc_tfidf).fit(tfidf_train, y_train)

# Model saving
with open('models/LinearSVCTFIDF.pickle', 'wb') as f:
    pickle.dump(model_svc_tfidf, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_svc_tfidf, model_name="Support Vector Classifier (Linear)", test_features=tfidf_test, test_labels=y_test, feature_vectorizer_name="TF-IDF")

print("Linear Support Vector Classifier (TF-IDF) sucessfully trained and stored.")

Linear Support Vector Classifier (TF-IDF) sucessfully trained and stored.


#### 3.6.2.4 With LIWC <a class="anchor" id="section_3_6_2_3"></a>

In [30]:
# Model

model_svc_liwc = LinearSVC(random_state=0)
# IF SAVED - Load saved model
#model_svc_liwc = pickle.load(open("models/LinearSVCLIWC.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Real(1, 15, name='C')] # Inverse of regularization strength

@use_named_args(space)
def objective(**params):
    model_svc_liwc.set_params(**params)

    return -np.mean(cross_val_score(model_svc_liwc, liwc_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_C_svc_liwc = res_gp.x[0]

print("""Best parameters: - C=%f""" % (best_C_svc_liwc))

Best parameters: - C=11.332275


In [19]:
# Model training with BEST hyper-parameteres

model_svc_liwc = LinearSVC(random_state=0, C=best_C_svc_liwc).fit(liwc_train, y_train)

# Model saving
with open('models/LinearSVCLIWC.pickle', 'wb') as f:
    pickle.dump(model_svc_liwc, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_svc_liwc, model_name="Support Vector Classifier (Linear)", test_features=liwc_test, test_labels=y_test, feature_vectorizer_name="LIWC")

print("Linear Support Vector Classifier (LIWC) sucessfully trained and stored.")

Linear Support Vector Classifier (LIWC) sucessfully trained and stored.


### 3.6.3 Multinomial Naive Bayes <a class="anchor" id="section_3_6_3"></a>

- Probabilistic approach to classifying documents in the case of acknowledging the frequency of a specified word in a text document;
- Achieves well on discrete types as the number of words found in a document;
- Conditional independence is assumed in real data and it attempts to approximate to the optimal soltuion;
- Is a quick classifier.

#### 3.6.3.1 With bag-of-words <a class="anchor" id="section_3_6_3_1"></a>

In [33]:
# Model 

model_nb_bow = MultinomialNB()
# IF SAVED - Load saved model
#model_nb_bow = pickle.load(open("models/MultinomialNaiveBayesBOW.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Real(0, 10, name='alpha')] # Additive (Laplace/Lidstone) smoothing parameter

@use_named_args(space)
def objective(**params):
    model_nb_bow.set_params(**params)

    return -np.mean(cross_val_score(model_nb_bow, bow_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_alpha_nb_bow = res_gp.x[0]

print("""Best parameters: - alpha=%f""" % (best_alpha_nb_bow))

Best parameters: - alpha=0.001813


In [20]:
# Model training with BEST hyper-parameteres

model_nb_bow = MultinomialNB(alpha=best_alpha_nb_bow).fit(bow_train, y_train)

# Model saving
with open('models/MultinomialNaiveBayesBOW.pickle', 'wb') as f:
    pickle.dump(model_nb_bow, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_nb_bow, model_name="Naive Bayes (Multinomial)", test_features=bow_test, test_labels=y_test, feature_vectorizer_name="Bag-of-words")

print("Multinomial Naive Bayes (Bag-of-words) successfully trained and stored.")

Multinomial Naive Bayes (Bag-of-words) successfully trained and stored.


#### 3.6.3.2 With TF-IDF <a class="anchor" id="section_3_6_3_2"></a>

In [36]:
# Model 

model_nb_tfidf = MultinomialNB()
# IF SAVED - Load saved model
#model_nb_tfidf = pickle.load(open("models/MultinomialNaiveBayesTFIDF.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Real(0, 10, name='alpha')] # Additive (Laplace/Lidstone) smoothing parameter

@use_named_args(space)
def objective(**params):
    model_nb_tfidf.set_params(**params)

    return -np.mean(cross_val_score(model_nb_tfidf, tfidf_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_alpha_nb_tfidf = res_gp.x[0]

print("""Best parameters: - alpha=%f""" % (best_alpha_nb_tfidf))

Best parameters: - alpha=0.338005


In [21]:
# Model training with BEST hyper-parameteres

model_nb_tfidf = MultinomialNB(alpha=best_alpha_nb_tfidf).fit(tfidf_train, y_train)

# Model saving
with open('models/MultinomialNaiveBayesTFIDF.pickle', 'wb') as f:
    pickle.dump(model_lr_tfidf, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_nb_tfidf, model_name="Naive Bayes (Multinomial)", test_features=tfidf_test, test_labels=y_test, feature_vectorizer_name="TF-IDF")

print("Multinomial Naive Bayes (TF-IDF) sucessfully trained and stored.")

Multinomial Naive Bayes (TF-IDF) sucessfully trained and stored.


#### 3.6.3.3 With LIWC <a class="anchor" id="section_3_6_3_3"></a>

In [39]:
# Model 

model_nb_liwc = MultinomialNB()
# IF SAVED - Load saved model
#model_nb_liwc = pickle.load(open("models/MultinomialNaiveBayesLIWC.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Real(0, 10, name='alpha')] # Additive (Laplace/Lidstone) smoothing parameter

@use_named_args(space)
def objective(**params):
    model_nb_liwc.set_params(**params)

    return -np.mean(cross_val_score(model_nb_liwc, liwc_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_alpha_nb_liwc = res_gp.x[0]

print("""Best parameters: - alpha=%f""" % (best_alpha_nb_liwc))

Best parameters: - alpha=0.000200


In [22]:
# Model training with BEST hyper-parameteres

model_nb_liwc = MultinomialNB(alpha=best_alpha_nb_liwc).fit(liwc_train, y_train)

# Model saving
with open('models/MultinomialNaiveBayesLIWC.pickle', 'wb') as f:
    pickle.dump(model_lr_liwc, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_nb_liwc, model_name="Naive Bayes (Multinomial)", test_features=liwc_test, test_labels=y_test, feature_vectorizer_name="LIWC")

print("Multinomial Naive Bayes (LIWC) sucessfully trained and stored.")

Multinomial Naive Bayes (LIWC) sucessfully trained and stored.


### 3.6.4 K-Nearest Neighbors <a class="anchor" id="section_3_6_4"></a>

- The data is classified based on vote among the k nearest neighbors;
- Number of neighbors can be sqrt(number of data objects) = int(sqrt(978)) = 31;
- Should be preferred when the data-set is relatively small.

#### 3.6.4.1 With bag-of-words <a class="anchor" id="section_3_6_4_1"></a>

In [45]:
# Model

model_knn_bow = KNeighborsClassifier() 
# IF SAVED - Load saved model
#model_knn_bow = pickle.load(open("models/KNNBOW.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Integer(1, 35, name='n_neighbors'), # Number of neighbors to use
          Categorical(["euclidean", "manhattan"], name='metric')] # The distance metric to use for the tree
            

@use_named_args(space)
def objective(**params):
    model_knn_bow.set_params(**params)

    return -np.mean(cross_val_score(model_knn_bow, bow_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_n_knn_bow = res_gp.x[0]
best_metric_knn_bow = res_gp.x[1]

print("Best parameters: - N_neighbors=%d Metric= %s" % (best_n_knn_bow, best_metric_knn_bow))

Best parameters: - N_neighbors=1 Metric= euclidean


In [23]:
# Model training with BEST hyper-parameteres

model_knn_bow = KNeighborsClassifier(n_neighbors=best_n_knn_bow, metric=best_metric_knn_bow).fit(bow_train, y_train)

# Model saving
with open('models/KNNBOW.pickle', 'wb') as f:
    pickle.dump(model_knn_bow, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_knn_bow, model_name="K-Nearest Neighbors", test_features=bow_test, test_labels=y_test, feature_vectorizer_name="Bag-of-words")

print("K-Nearest Neighbors (Bag-of-words) successfully trained and stored.")

K-Nearest Neighbors (Bag-of-words) successfully trained and stored.


#### 3.6.4.2 With TF-IDF <a class="anchor" id="section_3_6_4_2"></a>

In [48]:
# Model

model_knn_tfidf = KNeighborsClassifier()
# IF SAVED - Load saved model
#model_knn_tfidf = pickle.load(open("models/KNNTFIDF.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Integer(1, 35, name='n_neighbors'), # Number of neighbors to use
          Categorical(["euclidean", "manhattan"], name='metric')] # The distance metric to use for the tree
            

@use_named_args(space)
def objective(**params):
    model_knn_tfidf.set_params(**params)

    return -np.mean(cross_val_score(model_knn_tfidf, tfidf_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_n_knn_tfidf = res_gp.x[0]
best_metric_knn_tfidf = res_gp.x[1]

print("Best parameters: - N_neighbors=%d Metric= %s" % (best_n_knn_tfidf, best_metric_knn_tfidf))

Best parameters: - N_neighbors=33 Metric= euclidean


In [24]:
# Model training with BEST hyper-parameteres

model_knn_tfidf = KNeighborsClassifier(n_neighbors=best_n_knn_tfidf, metric=best_metric_knn_tfidf).fit(tfidf_train, y_train)

# Model saving
with open('models/KNNTFIDF.pickle', 'wb') as f:
    pickle.dump(model_knn_tfidf, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_knn_tfidf, model_name="K-Nearest Neighbors", test_features=tfidf_test, test_labels=y_test, feature_vectorizer_name="TF-IDF")

print("K-Nearest Neighbors (TF-IDF) successfully trained and stored.")

K-Nearest Neighbors (TF-IDF) successfully trained and stored.


#### 3.6.4.2 With LIWC <a class="anchor" id="section_3_6_4_2"></a>

In [51]:
# Model

model_knn_liwc = KNeighborsClassifier(n_neighbors=31, metric= 'euclidean')
# IF SAVED - Load saved model
#model_knn_liwc = pickle.load(open("models/KNNLIWC.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Integer(1, 35, name='n_neighbors'), # Number of neighbors to use
          Categorical(["euclidean", "manhattan"], name='metric')] # The distance metric to use for the tree
            

@use_named_args(space)
def objective(**params):
    model_knn_liwc.set_params(**params)

    return -np.mean(cross_val_score(model_knn_liwc, liwc_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_n_knn_liwc = res_gp.x[0]
best_metric_knn_liwc = res_gp.x[1]

print("Best parameters: - N_neighbors=%d Metric= %s" % (best_n_knn_liwc, best_metric_knn_liwc))

Best parameters: - N_neighbors=16 Metric= manhattan


In [25]:
# Model training with BEST hyper-parameteres

model_knn_liwc = KNeighborsClassifier(n_neighbors=best_n_knn_liwc, metric=best_metric_knn_liwc).fit(liwc_train, y_train)

# Model saving
with open('models/KNNLIWC.pickle', 'wb') as f:
    pickle.dump(model_knn_liwc, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_knn_liwc, model_name="K-Nearest Neighbors", test_features=liwc_test, test_labels=y_test, feature_vectorizer_name="LIWC")

print("K-Nearest Neighbors (LIWC) successfully trained and stored.")

K-Nearest Neighbors (LIWC) successfully trained and stored.


### 3.6.5 Random Forest <a class="anchor" id="section_3_6_5"></a>

- Fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.

#### 3.6.5.1 With bag-of-words <a class="anchor" id="section_3_6_5_1"></a>

In [54]:
# Model

model_rf_bow = RandomForestClassifier(criterion='entropy', random_state=0)
# IF SAVED - Load saved model
#model_rf_bow = pickle.load(open("models/RandomForestBOW.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Integer(10, 200, name='n_estimators'), # the number of trees in the forest
          Integer(3,100, name ='max_depth')] # the maximum allowable depth for each decision tree 

@use_named_args(space)
def objective(**params):
    model_rf_bow.set_params(**params)

    return -np.mean(cross_val_score(model_rf_bow, bow_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_nestimators_rf_bow = res_gp.x[0]
best_maxdepth_rf_bow = res_gp.x[1]

print("""Best parameters: - n_estimators=%d max_depth=%d""" % (best_nestimators_rf_bow, best_maxdepth_rf_bow))

Best parameters: - n_estimators=128 max_depth=40


In [26]:
# Model training with BEST hyper-parameteres

model_rf_bow = RandomForestClassifier(n_estimators=best_nestimators_rf_bow, max_depth=best_maxdepth_rf_bow, criterion='entropy', random_state=0).fit(bow_train, y_train)

# Model saving
with open('models/RandomForestBOW.pickle', 'wb') as f:
    pickle.dump(model_rf_bow, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_rf_bow, model_name="Random Forest", test_features=bow_test, test_labels=y_test, feature_vectorizer_name="Bag-of-words")

print("Random Forest (Bag-of-words) successfully trained and stored.")

Random Forest (Bag-of-words) successfully trained and stored.


#### 3.6.5.2 With TF-IDF <a class="anchor" id="section_3_6_5_2"></a>

In [57]:
# Model

model_rf_tfidf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
# IF SAVED - Load saved model
#model_rf_tfidf = pickle.load(open("models/RandomForestTFIDF.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Integer(100, 300, name='n_estimators'),
          Integer(3,30, name ='max_depth')] # the maximum allowable depth for each decision tree 

@use_named_args(space)
def objective(**params):
    model_rf_tfidf.set_params(**params)

    return -np.mean(cross_val_score(model_rf_tfidf, tfidf_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_nestimators_rf_tfidf = res_gp.x[0]
best_maxdepth_rf_tfidf = res_gp.x[1]

print("""Best parameters: - n_estimators=%d max_depth:%d""" % (best_nestimators_rf_tfidf, best_maxdepth_rf_tfidf))

Best parameters: - n_estimators=100 max_depth:25


In [27]:
# Model training with BEST hyper-parameteres

model_rf_tfidf = RandomForestClassifier(n_estimators=best_nestimators_rf_tfidf, max_depth=best_maxdepth_rf_tfidf, criterion='entropy', random_state=0).fit(tfidf_train, y_train)

# Model saving
with open('models/RandomForestTFIDF.pickle', 'wb') as f:
    pickle.dump(model_rf_tfidf, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_rf_tfidf, model_name="Random Forest", test_features=tfidf_test, test_labels=y_test, feature_vectorizer_name="TF-IDF")

print("Random Forest (TF-IDF) successfully trained and stored.")

Random Forest (TF-IDF) successfully trained and stored.


#### 3.6.5.3 With LIWC <a class="anchor" id="section_3_6_5_3"></a>

In [60]:
# Model

model_rf_liwc = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=0)
# IF SAVED - Load saved model
#model_rf_liwc = pickle.load(open("models/RandomForestLIWC.pickle", 'rb'))

# Hyper-parameteres tuning

space  = [Integer(100, 300, name='n_estimators'),
          Integer(3,30, name ='max_depth')] # the maximum allowable depth for each decision tree 

@use_named_args(space)
def objective(**params):
    model_rf_liwc.set_params(**params)

    return -np.mean(cross_val_score(model_rf_liwc, liwc_train, y_train, cv=5, n_jobs=-1, scoring="neg_mean_absolute_error"))

res_gp = gp_minimize(objective, space, random_state=0)

best_nestimators_rf_liwc = res_gp.x[0]
best_maxdepth_rf_liwc = res_gp.x[1]

print("""Best parameters: - n_estimators=%d max_depth:%d""" % (best_nestimators_rf_liwc, best_maxdepth_rf_liwc))

Best parameters: - n_estimators=181 max_depth:26


In [28]:
# Model training with BEST hyper-parameteres

model_rf_liwc = RandomForestClassifier(n_estimators=best_nestimators_rf_liwc, max_depth=best_maxdepth_rf_liwc, criterion='entropy', random_state=0).fit(liwc_train, y_train)

# Model saving
with open('models/RandomForestLIWC.pickle', 'wb') as f:
    pickle.dump(model_rf_liwc, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=model_rf_liwc, model_name="Random Forest", test_features=liwc_test, test_labels=y_test, feature_vectorizer_name="LIWC")

print("Random Forest (LIWC) successfully trained and stored.")

Random Forest (LIWC) successfully trained and stored.


## 3.7 Models training and optimization (Deep Learning) <a class="anchor" id="section_3_7"></a>

### 3.7.1 Convolutional Neural Network <a class="anchor" id="section_3_7_1"></a>

- Multi-layered artificial neural networks with the ability to detect complex features in data;
- Have been showing good results on text classification.

In [29]:
# Model

def create_cnn_model_for_tuning(hp):
    model_cnn_we = Sequential()

    # Add layers
    model_cnn_we.add(Embedding(vocabulary_length, 300, weights=[embedding_matrix], trainable=False, input_length=max_length_content))
    hp_rates = hp.Choice('rate', values=[0.2,0.3,0.4,0.5])
    model_cnn_we.add(Dropout(rate=hp_rates))
    hp_filters = hp.Int('filters', min_value = 10, max_value = 60)
    hp_kernels = hp.Int('kernel_size', min_value = 3, max_value = 15)
    model_cnn_we.add(Conv1D(filters=hp_filters, kernel_size=hp_kernels, padding="valid", activation="relu"))
    model_cnn_we.add(MaxPooling1D())
    model_cnn_we.add(Conv1D(filters=hp_filters, kernel_size=hp_kernels, padding="valid", activation="relu"))
    model_cnn_we.add(MaxPooling1D())
    model_cnn_we.add(Flatten())
    hp_units = hp.Int('units', min_value = 10, max_value = 100, step=10)
    model_cnn_we.add(Dense(units=hp_units, activation="relu"))
    model_cnn_we.add(Dropout(rate=hp_rates))
    model_cnn_we.add(Dense(1, activation="sigmoid")) # binary classification

    model_cnn_we.compile(optimizer='adam', loss='binary_crossentropy',  metrics=['accuracy'])
    return model_cnn_we

# IF SAVED - Load saved model
#model_cnn_we = pickle.load(open("models/CNNWE.pickle", 'rb'))

In [30]:
# Hyper-parameteres tuning

tuner = kt.Hyperband(create_cnn_model_for_tuning, objective='val_accuracy', max_epochs=5, factor=3, directory='hp_tuning', project_name='cnn')

tuner.search_space_summary()

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(we_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[stop_early])
best_hp=tuner.get_best_hyperparameters()[0]
best_model_cnn_we = tuner.hypermodel.build(best_hp)

best_model_cnn_we.summary()

Trial 10 Complete [00h 00m 42s]
val_accuracy: 0.9171974658966064

Best val_accuracy So Far: 0.9299362897872925
Total elapsed time: 00h 03m 19s
INFO:tensorflow:Oracle triggered exit


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 512, 300)          15385500  
                                                                 
 dropout_2 (Dropout)         (None, 512, 300)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 498, 44)           198044    
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 249, 44)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 235, 44)           29084     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 117, 44)          0         
 1D)                                                  

In [31]:
# Model training with BEST hyper-parameteres

best_model_cnn_we.fit(we_train, y_train, batch_size=32, epochs=10)

#plot_model(model_cnn_we, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# Model saving
with open('models/CNNWE.pickle', 'wb') as f:
    pickle.dump(best_model_cnn_we, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=best_model_cnn_we, model_name="Convolutional Neural Network", test_features=we_test, test_labels=y_test, feature_vectorizer_name="Word Embeddings", deep_learning=True)

print("CNN (Word Embeddings) successfully trained and stored.")

Epoch 1/10
25/25 [==============================] - 9s 336ms/step - loss: 0.5888 - accuracy: 0.6560
Epoch 2/10
25/25 [==============================] - 8s 313ms/step - loss: 0.2872 - accuracy: 0.8964
Epoch 3/10
25/25 [==============================] - 8s 312ms/step - loss: 0.2900 - accuracy: 0.9041
Epoch 4/10
25/25 [==============================] - 8s 311ms/step - loss: 0.1952 - accuracy: 0.9412
Epoch 5/10
25/25 [==============================] - 8s 312ms/step - loss: 0.1169 - accuracy: 0.9604
Epoch 6/10
25/25 [==============================] - 8s 312ms/step - loss: 0.0918 - accuracy: 0.9706
Epoch 7/10
25/25 [==============================] - 8s 316ms/step - loss: 0.0366 - accuracy: 0.9910
Epoch 8/10
25/25 [==============================] - 8s 312ms/step - loss: 0.0198 - accuracy: 0.9936
Epoch 9/10
25/25 [==============================] - 8s 313ms/step - loss: 0.0293 - accuracy: 0.9936
Epoch 10/10
25/25 [==============================] - 8s 318ms/step - loss: 0.0082 - accuracy: 0.9987

2022-04-22 22:14:56.528343: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://c4927962-46f1-459e-9a9f-99bbe9f94cd8/assets
CNN (Word Embeddings) successfully trained and stored.


### 3.7.2 Long Short-Term memory (LSTM)<a class="anchor" id="section_3_7_2"></a>

- Type of recurrent neural network that is better in terms of memory;
- Holds the required information and discard the information which is not required or useful for further prediction. 


In [32]:
# Model

def create_lstm_model_for_tuning(hp):
    model_lstm_we = Sequential()

    # Add layers
    model_lstm_we.add(Embedding(vocabulary_length, 300, weights=[embedding_matrix], trainable=False, input_length=max_length_content))
    model_lstm_we.add(LSTM(units=hp.Int('units',min_value=32, max_value=512, step=32), activation="sigmoid"))
    model_lstm_we.add(Dense(1, activation="sigmoid")) # binary classification

    model_lstm_we.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model_lstm_we

# IF SAVED - Load saved model
#model_lstm_we = pickle.load(open("models/LSTMWE.pickle", 'rb'))

In [33]:
# Hyper-parameteres tuning

tuner = kt.Hyperband(create_lstm_model_for_tuning, objective='val_accuracy', max_epochs=5, factor=3, directory='hp_tuning', project_name='lstm')

tuner.search_space_summary()

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(we_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[stop_early])
best_hp=tuner.get_best_hyperparameters()[0]
best_model_lstm_we = tuner.hypermodel.build(best_hp)

best_model_lstm_we.summary()

INFO:tensorflow:Reloading Oracle from existing project hp_tuning/lstm/oracle.json
INFO:tensorflow:Reloading Tuner from hp_tuning/lstm/tuner0.json
Search space summary
Default search space size: 1
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
INFO:tensorflow:Oracle triggered exit
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 512, 300)          15385500  
                                                                 
 lstm_1 (LSTM)               (None, 288)               678528    
                                                                 
 dense_5 (Dense)             (None, 1)                 289       
                                                                 
Total params: 16,064,317
Trainable params: 678,817
Non-trainable params: 15,385,500
___________________

In [34]:
# Model training with BEST hyper-parameteres

best_model_lstm_we.fit(we_train, y_train, batch_size=32, epochs=10)

# Model saving
with open('models/LSTMWE.pickle', 'wb') as f:
    pickle.dump(best_model_lstm_we, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=best_model_lstm_we, model_name="Long short-term memory (LSTM)", test_features=we_test, test_labels=y_test, feature_vectorizer_name="Word Embeddings", deep_learning=True)

print("LSTM (Word Embeddings) successfully trained and stored.")

Epoch 1/10
25/25 [==============================] - 67s 3s/step - loss: 0.6525 - accuracy: 0.6330
Epoch 2/10
25/25 [==============================] - 64s 3s/step - loss: 0.5681 - accuracy: 0.6893
Epoch 3/10
25/25 [==============================] - 62s 2s/step - loss: 0.4759 - accuracy: 0.7698
Epoch 4/10
25/25 [==============================] - 61s 2s/step - loss: 0.4001 - accuracy: 0.8133
Epoch 5/10
25/25 [==============================] - 61s 2s/step - loss: 0.3076 - accuracy: 0.8606
Epoch 6/10
25/25 [==============================] - 61s 2s/step - loss: 0.3216 - accuracy: 0.8645
Epoch 7/10
25/25 [==============================] - 61s 2s/step - loss: 0.3189 - accuracy: 0.8542
Epoch 8/10
25/25 [==============================] - 61s 2s/step - loss: 0.2325 - accuracy: 0.9169
Epoch 9/10
25/25 [==============================] - 61s 2s/step - loss: 0.2113 - accuracy: 0.9105
Epoch 10/10
25/25 [==============================] - 60s 2s/step - loss: 0.1998 - accuracy: 0.9207
INFO:tensorflow:Ass

LSTM (Word Embeddings) successfully trained and stored.


### 3.7.3 Bidirectional Long short-term memory (Bi-LSTM) <a class="anchor" id="section_3_7_3"></a>

- Bidirection: propagates the input forward and backwards through the RNN layer and then concatenates the final output.

In [35]:
# Model

def create_bilstm_model_for_tuning(hp):
    model_bilstm_we = Sequential()

    # Add layers
    model_bilstm_we.add(Embedding(vocabulary_length, 300, weights=[embedding_matrix], trainable=False, input_length=max_length_content))
    model_bilstm_we.add(Bidirectional(LSTM(units=hp.Int('units',min_value=32, max_value=512, step=32), activation="sigmoid")))
    model_bilstm_we.add(Dense(1, activation="sigmoid")) # binary classification

    model_bilstm_we.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model_bilstm_we

# IF SAVED - Load saved model
#model_bilstm_we = pickle.load(open("models/BILSTMWE.pickle", 'rb'))

In [36]:
# Hyper-parameteres tuning

tuner = kt.Hyperband(create_bilstm_model_for_tuning, objective='val_accuracy', max_epochs=5, factor=3, directory='hp_tuning', project_name='bilstm')

tuner.search_space_summary()

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
tuner.search(we_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[stop_early])
best_hp=tuner.get_best_hyperparameters()[0]
best_model_bilstm_we = tuner.hypermodel.build(best_hp)

best_model_bilstm_we.summary()

INFO:tensorflow:Reloading Oracle from existing project hp_tuning/bilstm/oracle.json


INFO:tensorflow:Reloading Oracle from existing project hp_tuning/bilstm/oracle.json


INFO:tensorflow:Reloading Tuner from hp_tuning/bilstm/tuner0.json


INFO:tensorflow:Reloading Tuner from hp_tuning/bilstm/tuner0.json


Search space summary
Default search space size: 1
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 512, 300)          15385500  
                                                                 
 bidirectional_1 (Bidirectio  (None, 704)              1838848   
 nal)                                                            
                                                                 
 dense_7 (Dense)             (None, 1)                 705       
                                                                 
Total params: 17,225,053
Trainable params: 1,839,553
Non-trainable params: 15,385,500
_________________________________________________________________


In [37]:
# Model training with BEST hyper-parameteres

best_model_bilstm_we.fit(we_train, y_train, batch_size=32, epochs=10)

# Model saving
with open('models/BILSTMWE.pickle', 'wb') as f:
    pickle.dump(best_model_bilstm_we, f)

# Store evaluation metrics
evaluation_metrics = evaluation(evaluation_metrics_list=evaluation_metrics, model=best_model_bilstm_we, model_name="Bidirectional Long short-term memory (Bi-LSTM)", test_features=we_test, test_labels=y_test, feature_vectorizer_name="Word Embeddings", deep_learning=True)

print("Bi-LSTM (Word Embeddings) successfully trained and stored.")

Epoch 1/10
25/25 [==============================] - 147s 6s/step - loss: 0.7034 - accuracy: 0.5473
Epoch 2/10
25/25 [==============================] - 144s 6s/step - loss: 0.6031 - accuracy: 0.6611
Epoch 3/10
25/25 [==============================] - 146s 6s/step - loss: 0.5172 - accuracy: 0.7353
Epoch 4/10
25/25 [==============================] - 147s 6s/step - loss: 0.4126 - accuracy: 0.8210
Epoch 5/10
25/25 [==============================] - 145s 6s/step - loss: 0.3283 - accuracy: 0.8555
Epoch 6/10
25/25 [==============================] - 146s 6s/step - loss: 0.2891 - accuracy: 0.8798
Epoch 7/10
25/25 [==============================] - 146s 6s/step - loss: 0.2559 - accuracy: 0.8951
Epoch 8/10
25/25 [==============================] - 148s 6s/step - loss: 0.2458 - accuracy: 0.8900
Epoch 9/10
25/25 [==============================] - 149s 6s/step - loss: 0.2166 - accuracy: 0.9194
Epoch 10/10
25/25 [==============================] - 149s 6s/step - loss: 0.1658 - accuracy: 0.9297
INFO:tens

INFO:tensorflow:Assets written to: ram://5285604a-8844-4836-9fa7-2755c9a8eaaa/assets


Bi-LSTM (Word Embeddings) successfully trained and stored.


## 3.8 Evaluation analysis <a class="anchor" id="section_3_8"></a>

The final evaluation (with the test dataset) is performed in the optimized models.

In [38]:
evaluation = pd.DataFrame(data=evaluation_metrics)
evaluation.columns = ['Model', 'Feature Vectorizer', 'Accuracy', 'Precision', 'Recall', 'F1']
evaluation = evaluation.sort_values(by='Accuracy', ascending=False)
evaluation

,Model,Feature Vectorizer,Accuracy,Precision,Recall,F1
7,Naive Bayes (Multinomial),TF-IDF,92.86,0.93,0.93,0.93
1,Logistic Regression,TF-IDF,92.86,0.93,0.93,0.93
12,Random Forest,Bag-of-words,92.35,0.92,0.92,0.92
0,Logistic Regression,Bag-of-words,91.84,0.92,0.92,0.92
13,Random Forest,TF-IDF,91.33,0.91,0.91,0.91
6,Naive Bayes (Multinomial),Bag-of-words,90.82,0.91,0.91,0.91
10,K-Nearest Neighbors,TF-IDF,90.31,0.90,0.90,0.90
17,Bidirectional Long short-term memory (Bi-LSTM),Word Embeddings,90.31,0.90,0.90,0.90
4,Support Vector Classifier (Linear),TF-IDF,90.31,0.90,0.90,0.90
3,Support Vector Classifier (Linear),Bag-of-words,89.80,0.90,0.90,0.90
